#   "Bibliotecas"

In [1]:
#"Bibliotecas"
import smtplib
import mimetypes
import pathlib
from email.mime.multipart import  MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Criando navegador e importando os dados

In [2]:
#Criar um navegador 
nav = webdriver.Chrome()

In [3]:
#Importar o banco de dados
pds = pd.read_excel("buscas.xlsx")
#=======================#
def df(BD):
    return display(BD)
#=======================#
df(pds)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,11 mini watch,2900,4800
1,rtx 3060,zota galax,2500,5000
2,playstation 5,4 3 FIFA,3000,6000


# def's 

#### - Analizar se ele n tem nenhum termo banidoa

In [4]:
def verivicar_termos_banidos(lis_termos_ban, nome):
    tem_tr_ban = False
    for palavra in lis_termos_ban:
        if palavra in nome:
            tem_tr_ban = True
    return tem_tr_ban

#### - Analizar se ele tem TODOS os termos PD

In [5]:
def verivicar_termos_protudo(lis_nomes_pd, nome):
    tem_td_pd = True
    for palavra in lis_nomes_pd:
        if palavra not in nome:
            tem_td_pd = False
    return tem_td_pd

#### - Busca no google Shopping

In [6]:
def busca_google_shopping(nav, prd, termos_banidos, preco_mi, preco_mx):
    # - Tratamentos 
    prd = prd.lower()
    termos_banidos = termos_banidos.lower()
    lis_termos_ban = termos_banidos.split(" ")
    lis_nomes_pd = prd.split(" ")
    preco_mi = float(preco_mi)
    preco_mx = float(preco_mx)
    lista_ru = []

    # - Entra no google
    nav.get("https://www.google.com/")
    nav.find_element('id','APjFqb').send_keys(prd,Keys.ENTER)
    
            # - Entrar na aba shopping
    elementos = nav.find_elements('class name','hdtb-mitem')
    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break
            # - Pegar as informações do produto
    lista_re = nav.find_elements('class name','i0X6df')

    for re in lista_re:    
        nome = re.find_element('class name','tAxDx').text.lower()
            # analizar se ele n tem nenhum termo banido
        tem_tr_ban = verivicar_termos_banidos(lis_termos_ban, nome)

            # analizar se ele tem TODOS os termos PD
        tem_td_pd = verivicar_termos_protudo(lis_nomes_pd, nome)

                # Seleciconar só os elementos que tem_tr_ban = False e au mesmo tempo tem_td_pd = True
        if not tem_tr_ban and tem_td_pd:
            # Tradar o preço
            preco = re.find_element('class name','a8Pemb').text
            if preco == '':
                preco = float(0)
            else:
                try:
                    preco = float(preco.replace("R$","").replace(" ","").replace(".","").replace(",","."))
                except:
                    continue

            # Verificar se o preço maximo esta entre preço maximo e minimo

            if preco_mi <= preco <= preco_mx:
                #=============================================#
                el_rf = re.find_element('class name','bONr3b')
                el_pai = el_rf.find_element('xpath','..')
                link = el_pai.get_attribute('href')
                #=============================================#    
                lista_ru.append((nome, preco, link))
    return lista_ru

#### - Busca no buscapé 

In [7]:
def busca_buscape(nav, prd, termos_banidos, preco_mi, preco_mx):
    # - Tratamentos 
    prd = prd.lower()
    termos_banidos = termos_banidos.lower()
    lis_termos_ban = termos_banidos.split(" ")
    lis_nomes_pd = prd.split(" ")
    preco_mi = float(preco_mi)
    preco_mx = float(preco_mx)
    lista_ru = []
    
    # - Entra no google
    nav.get("https://www.buscape.com.br/")
    nav.find_element('class name','AutoCompleteStyle_input__HG105').send_keys(prd,Keys.ENTER)
    # - Pegar as informações do produto
    
    while len(nav.find_elements('class name','Select_Select__1S7HV')) < 1:
        time.sleep(1)
        
    lista_re = nav.find_elements('class name','SearchCard_ProductCard_Inner__7JhKb')
    
    for re in lista_re:
        preco = re.find_element('class name','Text_MobileHeadingS__Zxam2').text
        nome = re.find_element('class name','SearchCard_ProductCard_Name__ZaO5o').text.lower()
       
        link = re.get_attribute("href")
            # analizar se ele n tem nenhum termo banido
        tem_tr_ban = verivicar_termos_banidos(lis_termos_ban, nome)

            # analizar se ele tem TODOS os termos PD
        tem_td_pd = verivicar_termos_protudo(lis_nomes_pd, nome)
        
        if not tem_tr_ban and tem_td_pd:
            
            # Tradar o preço
            if preco == '':
                preco = float(0)
            else:
                preco = float(preco.replace("R$","").replace(" ","").replace(".","").replace(",","."))
                
            if preco_mi <= preco <= preco_mx:  
                lista_ru.append((nome, preco, link))
    return lista_ru

# O "Codigo"

In [8]:
tabela_ofertas  = pd.DataFrame()

for linha in pds.index:
    prd = pds.loc[linha,"Nome"]
    termos_banidos = pds.loc[linha,"Termos banidos"]
    preco_mi = pds.loc[linha,"Preço mínimo"]
    preco_mx = pds.loc[linha,"Preço máximo"]
    
    lista_ru_gshop = busca_google_shopping(nav, prd, termos_banidos,preco_mi,preco_mx)
    if lista_ru_gshop:
        tabela_google = pd.DataFrame(lista_ru_gshop, columns=["Protudo", "Preço", "Link"])
        tabela_ofertas  = pd.concat([tabela_ofertas,tabela_google])
    else:
        tabela_google = None
    
    lista_ru_buscape = busca_buscape(nav, prd, termos_banidos, preco_mi, preco_mx)
    if lista_ru_buscape:
        tabela_buscape = pd.DataFrame(lista_ru_buscape, columns=["Protudo", "Preço", "Link"])
        tabela_ofertas  = pd.concat([tabela_ofertas,tabela_buscape])
    else:
        tabela_buscape = None
#df(tabela_ofertas)

# Exporta a tabela de ofertas para o excel

In [9]:
tabela_ofertas.to_excel("ofertas.xlsx", index=False)

# Enviar o E-mail com a tabela oferta

In [10]:
if len(tabela_ofertas) > 0:
    with open('Senha.txt','r') as arquivo:
        senha = arquivo.read()
        
        msg= MIMEMultipart()

        msg['Subject'] = "Faixas de preços."
        msg['From'] = 'testes14lii@gmail.com'
        msg['To'] = 'testes14lii@gmail.com'

        corpo_email = """Protudo(s) encontrado(s) na faxa de preço solicidada."""
        
        msg.attach(MIMEText(corpo_email,'html'))
        local_aq = r"C:\Users\Censu\Documents\jupyter\Projeto - 2 - Automação web\ofertas.xlsx"
        
        filename = 'ofertas.xlsx'
        attachment = open(local_aq, "rb")

        anexo_type = mimetypes.guess_type(f'{filename}')[0].split('/')
        part = MIMEBase(anexo_type[0], anexo_type[1])

        part.set_payload((attachment).read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition',"attachment; filename= %s" % filename)

        msg.attach(part)

        s = smtplib.SMTP('smtp.gmail.com: 587')
        s.starttls()

        #login Credentials for sending the mail
        s.login(msg['From'], senha)
        s.sendmail(msg['From'], msg['To'], msg.as_string())
        s.quit()
        print(f"E-mail enviado com Sucesso")

E-mail enviado com Sucesso
